# Ciminion DRL Gröbner Basis
DRL Gröbner basis computation for an arbitrary Ciminion instance.

In [1]:
load("Ciminion.sage")
load("Ciminion_polynomial_model.sage")

## Ciminion Instance

In [2]:
ciminion = Ciminion(info_level=1)

Ciminion parameters
Field: Finite Field of size 170141183460469231731687303715884105773
Rounds_C: 90
Rounds_E: 14
Constants_C: [[12443888061132564955549422541438338816, 21276614716541696695318086670097828502, 120078163218193667153684845643983415685, 130641005060990866193616249940757591750], [125887459707080344689085893495418321703, 71902286960901024440532316867644209659, 120910965005320520908694143787110551307, 97797964509982263506823135581495489116], [132865193887641769457802614559975422926, 38488086883189222302460794339043471872, 29706156733030754534529182976908311722, 160327256483367825045776479872441910139], [79642928695301517089431370050163876697, 66799797625571595833108116835078520750, 166338163794335552416229755317929724333, 65504095875764583110334854922655768975], [148846746057683853298397085740800393026, 28300719801627152379400077167055246215, 120976720741821249275637951253712710909, 81571275723970086981306045729594197170], [13778046550869800057852992214688502555, 980348508076

## Ciminion Polynomial model

In [3]:
polys = generate_Ciminion_polynomials(ciminion=ciminion)

P = polys[0].parent()

Plaintext: [32800612652959847124316514589726708264, 145659599728499720709638026446796571940]
Key: [85314934235486398593819925056549730156, 88814908100800986011574143271891845963]
Nonce: 54148340874303256744603691668450571124
Ciphertext: [150257598618921469634611785749844327004, 150469368259954595160326754157798976285]
Term order: degrevlex


## DRL Gröbner Basis computation
First we change to a different DRL term order.

In [4]:
f = efficient_Ciminion_termorder(ciminion, P)

Next we invert the matrices in the round functions, extract the linear polynomials in each round, and perform Gaussian elimination on the linear polynomials.

In [5]:
polys = [f(poly) for poly in transform_Ciminion_polynomial_system(ciminion, polys)]

lin_polys = Sequence([poly for poly in polys if poly.degree() == 1])
M, v = lin_polys.coefficients_monomials()

M = M.echelon_form()

lin_polys = ideal((M * v).list())

Now we reduce the non-linear polynomials with respect to the linear polynomials, i.e. we eliminate variables in the non-linear polynomials.
We call the resulting polynomial system the downsized Gröbner basis.

In [6]:
gb_downsized = [poly.reduce(lin_polys) for poly in polys if poly.degree() > 1]
lms = [poly.lm() for poly in gb_downsized]

In [7]:
for mon in lms:
    print(mon)

x_C_3__2^2
x_C_3__3^2
x_C_3__4^2
x_C_3__5^2
x_C_3__6^2
x_C_3__7^2
x_C_3__8^2
x_C_3__9^2
x_C_3__10^2
x_C_3__11^2
x_C_3__12^2
x_C_3__13^2
x_C_3__14^2
x_C_3__15^2
x_C_3__16^2
x_C_3__17^2
x_C_3__18^2
x_C_3__19^2
x_C_3__20^2
x_C_3__21^2
x_C_3__22^2
x_C_3__23^2
x_C_3__24^2
x_C_3__25^2
x_C_3__26^2
x_C_3__27^2
x_C_3__28^2
x_C_3__29^2
x_C_3__30^2
x_C_3__31^2
x_C_3__32^2
x_C_3__33^2
x_C_3__34^2
x_C_3__35^2
x_C_3__36^2
x_C_3__37^2
x_C_3__38^2
x_C_3__39^2
x_C_3__40^2
x_C_3__41^2
x_C_3__42^2
x_C_3__43^2
x_C_3__44^2
x_C_3__45^2
x_C_3__46^2
x_C_3__47^2
x_C_3__48^2
x_C_3__49^2
x_C_3__50^2
x_C_3__51^2
x_C_3__52^2
x_C_3__53^2
x_C_3__54^2
x_C_3__55^2
x_C_3__56^2
x_C_3__57^2
x_C_3__58^2
x_C_3__59^2
x_C_3__60^2
x_C_3__61^2
x_C_3__62^2
x_C_3__63^2
x_C_3__64^2
x_C_3__65^2
x_C_3__66^2
x_C_3__67^2
x_C_3__68^2
x_C_3__69^2
x_C_3__70^2
x_C_3__71^2
x_C_3__72^2
x_C_3__73^2
x_C_3__74^2
x_C_3__75^2
x_C_3__76^2
x_C_3__77^2
x_C_3__78^2
x_C_3__79^2
x_C_3__80^2
x_C_3__81^2
x_C_3__82^2
x_C_3__83^2
x_C_3__84^2
x_C_3__85^2


In [8]:
gcds = [gcd(comb[0], comb[1]) for comb in Combinations(lms, 2)]
list(set(gcds))

[1]

Since all leading monomials are pairwise coprime we have indeed constructed a DRL Gröbner basis.

Finally, let us verify that the downsized Gröbner basis is indeed zero_dimensional by extracting the variables present in the downsized system.

In [9]:
P = gb_downsized[0].parent()
variables = P.gens()
eliminated_variables = [poly.lm() for poly in lin_polys.gens()]
variables_downsized = [var for var in variables if not var in eliminated_variables]

In [10]:
ideal([var**2 for var in variables_downsized]) == ideal(lms)

True

In [11]:
M, v = Sequence(gb_downsized).coefficients_monomials()
M = M.echelon_form()

gb_downsized_red = list((vector(M * v)))

In [12]:
sep = 100 * "-"

for poly in gb_downsized_red:
    print(poly)
    print(sep)

x_C_3__2^2 + 36052268501098691480782022221440140193*x_C_3__2*x_C_3__3 + 78016920990122962277968399409826175480*x_C_3__2*x_C_3__4 + 42703712258299046583688411921407572776*x_C_3__2*x_C_3__5 + 38442538836974364768003122300339781971*x_C_3__2*x_C_3__6 + 108523168302645023986238152135441735593*x_C_3__2*x_C_3__7 + 115302224439152063435950142961689206128*x_C_3__2*x_C_3__8 + 154657176015297785859113181162685924288*x_C_3__2*x_C_3__9 + 13866325932474051578841929255506338028*x_C_3__2*x_C_3__10 + 90181469822206232989215734520214580973*x_C_3__2*x_C_3__11 + 92084822327292320090016061801288877145*x_C_3__2*x_C_3__12 + 157354278602989357347956240137892176486*x_C_3__2*x_C_3__13 + 97731795234864588111455071197124577425*x_C_3__2*x_C_3__14 + 78292514079536238154193684691293671862*x_C_3__2*x_C_3__15 + 65994669206827023258952250101859132184*x_C_3__2*x_C_3__16 + 144921306020444886667113891963386487131*x_C_3__2*x_C_3__17 + 92416065826963421097657471806457189391*x_C_3__2*x_C_3__18 + 49752721776763552889121710807

In [13]:
set(flatten([poly.variables() for poly in gb_downsized_red])) == set(flatten([poly.lm().variables() for poly in gb_downsized_red]))

True